In [1]:
# Standard packages
import tempfile
import warnings
import urllib
import shutil
import os
# Less standard, but still pip- or conda-installable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
#import rasterio
import re
import rtree
import shapely
import pickle
import data_eng.az_proc as ap
#from cartopy import crs
import collections
import cv2
import math

# Confirm Tiles Are Downloads (Or Download Tiles)

In [2]:
sub_directories
img_annotation_path 

#Identify the tiles that the images come from
tile_id = []
for image in os.listdir(os.path.join(parent,annotation_directory,anno)): #iterate through the images
    tile_id.append(image.rsplit("_",1)[0])
tile_id = np.unique(np.array(tile_id))
#print(tile_id)

#identify the tiles that are already downloaded
downloaded_tiles = []
for tiles in os.listdir(os.path.join(parent,annotation_directory,"tiles")): #iterate through the images
    downloaded_tiles.append(tiles.split(".",1)[0])
downloaded_tiles= np.unique(np.array([downloaded_tiles]))
#print(downloaded_tiles)

tiles_labeled = "tile_name_tile_url_labeled.npy"
np_tiles_labeled = np.load(tiles_labeled)
for labeled_image_tile_id in tile_id: # the tiles from images 
    if labeled_image_tile_id not in tiles: #the downloaded times 
        #print(labeled_image_tile_id)
        #download the tiles if they are not in the tiles folder 
        for tile_name_url in np_tiles_labeled[:,]:
            if labeled_image_tile_id == tile_name_url[0].split(".",1)[0]:
                ###Download tile
                ap.download_url(tile_name_url[1], os.path.join(parent,annotation_directory,"tiles"),
                                                         progress_updater=ap.DownloadProgressBar())


NameError: name 'sub_directories' is not defined

In [4]:
parent_directory = "C:\chips_allocation"
annotation_directory = "Check"
number_of_tiles =10
tiles_remaining = "tile_name_tile_url_remaining_expanded.npy"
tiles_labeled = "tile_name_tile_url_labeled.npy"

Unlabeled Tiles (801, 2)


In [ ]:
dist.chip_tiles()

function_opened
ca_060cm_2018_38121_m_3812128_ne_10_060_20180714.tif
12220 9770
23 19
24 20
481
ca_060cm_2018_38122_m_3812263_ne_10_060_20180723.tif
12150 9750
23 19
24 20
481
ct_060cm_2018_42072_m_4207260_se_18_060_20180823.tif
12817 9958
25 19
26 20
521
ky_060cm_2018_37087_m_3708712_nw_16_060_20180725.tif
12523 10311
24 20
25 21
526
mi_060cm_2018_43083_m_4308317_sw_17_060_20180703.tif
12965 9885
25 19
26 20
521
nm_060cm_2018_32103_m_3210330_nw_13_060_20180618.tif
12210 10470
23 20
24 21
505
tx_060cm_2018_30101_m_3010109_ne_14_060_20190105.tif
12320 10780
24 21
25 22
551
tx_060cm_2018_32095_m_3209516_se_15_060_20181120.tif
12250 10490
23 20
24 21
505
ut_060cm_2018_40111_m_4011109_se_12_060_20180828.tif
12160 9420
23 18
24 19
457
wi_060cm_2018_46092_m_4609224_ne_15_060_20181022.tif
12674 9105
24 17
25 18
451


In [19]:
tile.shape[0] #Height


12220